# Processing an eDNA dataset to Darwin Core

<div class="alert alert-warning">
If you are new yo R, make sure to read these info boxes.
</div>

First load some dependencies and create the output directory:

In [1]:
library(dplyr)
library(readxl)

# make sure to change the output directory to your own
output_dir <- "../dwc/pieter"
dir.create(output_dir)

# limit number of rows in notebook output
options(repr.matrix.max.rows = 10, repr.matrix.max.cols = 20)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




<div class="alert alert-warning">
In R, packages can be loaded using <code>library()</code>. If a package is not installed, you can install it from CRAN using <code>install.packages()</code>. The <code>dplyr</code> package is a commonly used package for data manipulation, <code>readxl</code> is required for reading the Excel file.
</div>

## Reading the original dataset
### List all dataset files

In [2]:
list.files("../dataset", full.names = "TRUE")

[1] "../dataset/metadata.txt"    "../dataset/samples.xlsx"   
[3] "../dataset/seqtab.txt"      "../dataset/sequences.fasta"
[5] "../dataset/taxonomy.txt"

<div class="alert alert-warning">
In a relative file path, <code>..</code> indicates the parent directory.
</div>

### Read the ASV table

`../dataset/seqtab.txt` contains the ASV table, so it has one row per ASV, and the number of reads in a sample in different columns.

In [3]:
seqtab <- read.table("../dataset/seqtab.txt", sep = "\t", header = TRUE)
seqtab

asv,EE0493,EE0495
<chr>,<int>,<int>
asv.1,0,0
asv.2,14,2447
asv.3,0,0
asv.4,0,0
asv.5,40587,1857
⋮,⋮,⋮
asv.16981,0,0
asv.16982,0,0
asv.16983,0,0


<div class="alert alert-warning">
<code>read.table()</code> reads a delimited text file to a data frame. <code>sep = "\t"</code> means that the file is tab delimited.
</div>

### Read the taxonomy file

`../dataset/taxonomy.txt` contains a taxon name for each ASV.

In [4]:
taxonomy <- read.table("../dataset/taxonomy.txt", sep = "\t", header = TRUE)
taxonomy

asv,taxonomy
<chr>,<chr>
asv.1,Eukaryota
asv.2,Clausocalanus_furcatus
asv.3,Eurotatoria
asv.4,Arthropoda
asv.5,Eukaryota
⋮,⋮
asv.16981,Metazoa
asv.16982,Metazoa
asv.16983,Metazoa


These names originate from the reference database and will have to be matched to WoRMS later.

### Read the sample metadata

We also have an Excel file with sample info.

In [5]:
samples <- read_excel("../dataset/samples.xlsx")
samples

name,size,event_begin,area_name,area_longitude,area_latitude,area_uncertainty,parent_area_name,dna,depth,temperature
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
EE0493,1450,24/04/2023,Ile esprit,46.22536,9.425180,20,Aldabra Atoll,7.23,10,26.3
EE0495,1500,02/04/2023,Settlement beach,46.20605,9.400901,98,Aldabra Atoll,15.83,12,25.1


## Joining the tables

At this point we could start quality control on the individual tables, but if we first join and map the tables to Darwin Core occurrence terms, the quality control code will be easier to read.

### Event fields

Let's start with the sample table. This table has sample identifiers, time, coordinates, coordinate uncertainty, locality, and higher geography which can all be mapped to Darwin Core. Keep the `dna` field for later.

In [ ]:
event <- samples %>%
    select(
        eventID = name,
        materialSampleID = name,
        eventDate = event_begin,
        locality = area_name,
        decimalLongitude = area_longitude,
        decimalLatitude = area_latitude,
        coordinateUncertaintyInMeters = area_uncertainty,
        higherGeography = parent_area_name,
        minimumDepthInMeters = depth,
        maximumDepthInMeters = depth,
        sampleSizeValue = size,
        dna,
        temperature
    ) %>%
    mutate(sampleSizeUnit = "ml")
event

<div class="alert alert-warning">
The code above uses several <code>dlyr</code> functions. <code>select()</code> selects and optionally renames a set of columns from the dataframe. <code>mutate()</code> creates a new column. <code>%>%</code> is the pipe operator which is used to string functions together.
</div>

### Occurrence fields

Next is the ASV table. This table is in a wide format with ASVs as rows and samples as columns. We will convert this to a long format, with one row per occurrence and the number of sequence reads as `organismQuantity`. We will use the sample identifier as `eventID` and the combination of sample identifier and ASV number as the `occurrenceID`.

<div class="alert alert-warning">
To do from a wide to a long table, use the <code>gather()</code> function from the <code>tidyr</code> package. <code>paste0()</code> is used to combine character strings.

In [6]:
library(tidyr)

occurrence <- seqtab %>%
    gather(eventID, organismQuantity, 2:3) %>%
    filter(organismQuantity > 0) %>%
    mutate(
        occurrenceID = paste0(eventID, "_", asv),
        organismQuantityType = "sequence reads"
    )
occurrence

asv,eventID,organismQuantity,occurrenceID,organismQuantityType
<chr>,<chr>,<int>,<chr>,<chr>
asv.2,EE0493,14,EE0493_asv.2,sequence reads
asv.5,EE0493,40587,EE0493_asv.5,sequence reads
asv.6,EE0493,7,EE0493_asv.6,sequence reads
asv.7,EE0493,29367,EE0493_asv.7,sequence reads
asv.8,EE0493,72378,EE0493_asv.8,sequence reads
⋮,⋮,⋮,⋮,⋮
asv.16949,EE0495,1,EE0495_asv.16949,sequence reads
asv.16958,EE0495,1,EE0495_asv.16958,sequence reads
asv.16961,EE0495,1,EE0495_asv.16961,sequence reads


We can now add the taxonomic names to our occurrence table.

In [7]:
taxonomy <- taxonomy %>%
    select(asv, verbatimIdentification = taxonomy)

In [ ]:
occurrence <- occurrence %>%
    left_join(taxonomy, by = "asv")
occurrence

<div class="alert alert-warning">
<code>left_join()</code> joins two dataframes by matching columns. The <code>by</code> argument specifies the columns to match on.
</div>

### Joining event and occurrence fields

In [ ]:
occurrence <- event %>%
    left_join(occurrence, by = "eventID")
occurrence

## Adding metadata

Populate `samplingProtocol` with a link the the eDNA Expeditions protocol.

In [ ]:
occurrence$samplingProtocol <- "https://github.com/BeBOP-OBON/UNESCO_protocol_collection"

## Quality control

### Taxon matching

Let's first match the taxa with WoRMS. This can be done using the `obistools` package. Before matching with WoRMS we will remove underscores from the scientific names.

In [ ]:
taxon_names <- stringr::str_replace(occurrence$verbatimIdentification, "_", " ")

Now match the names, this can take a few minutes.

In [ ]:
matched <- obistools::match_taxa(taxon_names, ask = FALSE) %>%
    select(scientificName, scientificNameID)

matched

In [ ]:
occurrence <- bind_cols(occurrence, matched)
occurrence

In [ ]:
non_matches <- occurrence %>%
    filter(is.na(scientificNameID)) %>%
    group_by(verbatimIdentification) %>%
    summarize(n = n()) %>%
    arrange(desc(n))

write.table(non_matches, file = file.path(output_dir, "nonmatches.txt"), sep = "\t", row.names = FALSE, na = "", quote = FALSE)

non_matches

Normally we have to resolve these names one by one, but for this exercise we will just fix the most common errors. For example, records annotated as eukaryotes can be populated with scientificName `Incertae sedis` and scientificNameID `urn:lsid:marinespecies.org:taxname:12`.

In [ ]:
occurrence <- occurrence %>%
    mutate(
        scientificName = case_when(verbatimIdentification %in% c("Eukaryota", "undef_Eukaryota", "") ~ "Incertae sedis", .default = scientificName),
        scientificNameID = case_when(verbatimIdentification %in% c("Eukaryota", "undef_Eukaryota", "") ~ "urn:lsid:marinespecies.org:taxname:12", .default = scientificNameID)
    )

In [ ]:
occurrence %>%
    filter(is.na(scientificNameID)) %>%
    group_by(verbatimIdentification) %>%
    summarize(n = n()) %>%
    arrange(desc(n))

In [ ]:
occurrence

### Location

Now let's check the coordinates by plotting the distinct coordinate pairs on a map.

In [ ]:
library(leaflet)

stations <- occurrence %>%
    distinct(locality, decimalLongitude, decimalLatitude)

stations

leaflet() %>%
    addTiles() %>%
    addMarkers(lng = stations$decimalLongitude, lat = stations$decimalLatitude, popup = stations$locality)

There's clearly something wrong with the coordinates. Longitude looks fine, let's try flipping latitude.

In [ ]:
occurrence <- occurrence %>%
    mutate(decimalLatitude = -decimalLatitude)

stations <- occurrence %>%
    distinct(locality, decimalLongitude, decimalLatitude)
stations

leaflet() %>%
    addTiles() %>%
    addMarkers(lng = stations$decimalLongitude, lat = stations$decimalLatitude, popup = stations$locality)

Now fix the occurrence table.

### Time

Now check the event dates.

In [ ]:
obistools::check_eventdate(occurrence)

It looks like `eventDate` is in the wrong format. Use the `lubridate` package to parse the current date format and change it.

In [ ]:
library(lubridate)

occurrence <- occurrence %>%
    mutate(eventDate = format_ISO8601(parse_date_time(eventDate, "%d/%m/%Y"), precision = "ymd", usetz = FALSE))

unique(occurrence$eventDate)

In [ ]:
head(occurrence)

### Missing fields

Let's check if any required fields are missing.

In [ ]:
obistools::check_fields(occurrence)

In [ ]:
occurrence <- occurrence %>%
    mutate(
        occurrenceStatus = "present",
        basisOfRecord = "MaterialSample"
    )

## MeasurementOrFact

We have several measurements that can be added to the MeasurementOrFact extension: sequence reads, sample volume, and DNA extract concentration.

In [ ]:
mof_reads <- occurrence %>%
    select(occurrenceID, measurementValue = organismQuantity) %>%
    mutate(
        measurementType = "sequence reads"
    )

mof_samplesize <- occurrence %>%
    select(occurrenceID, measurementValue = sampleSizeValue, measurementUnit = sampleSizeUnit) %>%
    mutate(
        measurementType = "sample size",
        measurementTypeID = "http://vocab.nerc.ac.uk/collection/P01/current/VOLWBSMP/",
        measurementUnit = "ml",
        measurementUnitID = "http://vocab.nerc.ac.uk/collection/P06/current/VVML/"
    )

mof_dna <- occurrence %>%
    select(occurrenceID, measurementValue = dna) %>%
    mutate(
        measurementType = "DNA concentration",
        measurementTypeID = "http://vocab.nerc.ac.uk/collection/P01/current/A260DNAX/",
        measurementUnit = "ng/μl",
        measurementUnitID = "http://vocab.nerc.ac.uk/collection/P06/current/UNUL/"
    )

mof_temperature <- occurrence %>%
    select(occurrenceID, measurementValue = temperature) %>%
    mutate(
        measurementType = "seawater temperature",
        measurementTypeID = "http://vocab.nerc.ac.uk/collection/P01/current/TEMPPR01/",
        measurementUnit = "degrees Celsius",
        measurementUnitID = "http://vocab.nerc.ac.uk/collection/P06/current/UPAA/"
    )

mof <- bind_rows(mof_reads, mof_samplesize, mof_dna)
mof

## DNADerivedData
### Reading sequence data

In [ ]:
library(Biostrings)

fasta_file <- readDNAStringSet("../dataset/sequences.fasta")
fasta <- data.frame(asv = names(fasta_file), DNA_sequence = paste(fasta_file))
fasta

In [ ]:
dna <- occurrence %>%
    select(occurrenceID, asv, concentration = dna) %>%
    left_join(fasta, by = "asv")

dna

### Adding metadata

We have a file with some sequencing metadata, print the file contents and add the corresponding fields to the DNADerivedData table.

In [ ]:
cat(paste0(readLines("../dataset/metadata.txt"), collapse = "\n"))

In [ ]:
dna <- dna %>%
    mutate(
        concentrationUnit = "ng/μl",
        lib_layout = "paired",
        target_gene = "COI",
        pcr_primers = "FWD:GGWACWGGWTGAACWGTWTAYCCYCC;REV:TANACYTCNGGRTGNCCRAARAAYCA",
        seq_meth = "Illumina NovaSeq6000",
        ref_db = "https://github.com/iobis/edna-reference-databases",
        pcr_primer_forward = "GGWACWGGWTGAACWGTWTAYCCYCC",
        pcr_primer_reverse = "TANACYTCNGGRTGNCCRAARAAYCA",
        pcr_primer_name_forward = "mlCOIintF",
        pcr_primer_name_reverse = "dgHCO2198",
        pcr_primer_reference = "doi:10.1186/1742-9994-10-34"
    ) %>%
    select(-asv)

dna

## Output

Write text files and compress.

In [ ]:
occurrence <- occurrence %>%
    select(-asv, -dna, -temperature)

write.table(occurrence, file = file.path(output_dir, "occurrence.txt"), sep = "\t", row.names = FALSE, na = "", quote = FALSE)
write.table(mof, file = file.path(output_dir, "measurementorfact.txt"), sep = "\t", row.names = FALSE, na = "", quote = FALSE)
write.table(dna, file = file.path(output_dir, "dnaderiveddata.txt"), sep = "\t", row.names = FALSE, na = "", quote = FALSE)

In [ ]:
library(dwcawriter)

archive <- list(
    eml = '<eml:eml packageId="https://obis.org/dummydataset/v1.0" scope="system" system="http://gbif.org" xml:lang="en" xmlns:dc="http://purl.org/dc/terms/" xmlns:eml="eml://ecoinformatics.org/eml-2.1.1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="eml://ecoinformatics.org/eml-2.1.1 http://rs.gbif.org/schema/eml-gbif-profile/1.2/eml.xsd">
        <dataset>
        <title xml:lang="en">Dummy Dataset</title>
        </dataset>
    </eml:eml>',
    core = list(
        name = "occurrence",
        type = "https://rs.gbif.org/core/dwc_occurrence_2022-02-02.xml",
        index = which(names(occurrence) == "occurrenceID"),
        data = occurrence
    ),
    extensions = list(
        list(
            name = "measurementorfact",
            type = "https://rs.gbif.org/extension/obis/extended_measurement_or_fact_2023-08-28.xml",
            index = which(names(mof) == "occurrenceID"),
            data = mof
        ),
        list(
            name = "dnaderiveddata",
            type = "https://rs.gbif.org/extension/gbif/1.0/dna_derived_data_2022-02-23.xml",
            index = which(names(dna) == "occurrenceID"),
            data = dna
        )
    )
)

write_dwca(archive, file.path(output_dir, "archive.zip"))